In [74]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy
import sklearn
from math import *
from prettytable import PrettyTable
from functools import partial
from decimal import Decimal
import sympy

plt.rcParams["font.sans-serif"] = ["Microsoft YaHei"]
plt.rcParams["axes.unicode_minus"] = False  # 解决无法显示符号的问题
palette = "deep"
sns.set(font="Microsoft YaHei", font_scale=1.2, palette=palette)  # 解决Seaborn中文显示问题

In [75]:
from mcm_cn2023.utils.util import *

In [76]:
def getFns(
    起始点: tuple, 南北长度: float, 东西长度: float, 换能器开角: float, 坡度: float, 海域中心点水深: float
):
    """

    所有角度入参都是弧度制！！！函数内部不做转换

    Arguments:

    起始点 - 单位米，例如：(0, 0)

    南北长度 - 单位米

    东西长度 - 单位米

    换能器开角 - 单位弧度制

    坡度 - 单位弧度制

    Returns:
    """
    起始点x, 起始点y = 起始点

    alpha = 坡度
    theta = 换能器开角

    """
    东^ 浅
    A----------B------> y方向
    |          |
    |          |
    |          |
    |          |
    |          |
    D----------C
    |
    |
    x方向
    
    西 深
    """
    AD = 东西长度
    AB = 南北长度

    p待测区A = sympy.Point2D(0, 0)
    p待测区B = sympy.Point2D(0, AB)
    p待测区C = sympy.Point2D(AD, AB)
    p待测区D = sympy.Point2D(AD, 0)

    def 根据参考点计算相对点的海深度(参考点深度, 测量船距离参考点, 测线方向夹角):
        beta = 测线方向夹角
        d = 测量船距离参考点
        d0 = 参考点深度
        val = d0 + (d * (cos(beta) * sin(alpha))) / sqrt(
            1 - cos(beta) ** 2 * sin(alpha) ** 2
        )
        return val

    def 计算覆盖宽度(测线方向夹角, 测量船距离参考点, 参考点深度):
        def gamma(beta):
            val = asin(abs(sin(alpha * sin(beta))))
            return val

        gammar = gamma(测线方向夹角)
        deep = 根据参考点计算相对点的海深度(参考点深度, 测量船距离参考点, 测线方向夹角)
        val = (
            deep
            * cos(gammar)
            * sin(theta)
            / (cos(gammar - theta / 2) * cos(gammar + theta / 2))
        )
        return val

    def 计算测线长度(测线方向夹角):
        beta = 测线方向夹角
        betaQuote = atan(东西长度 / 南北长度)
        if 测线方向夹角 < betaQuote:
            return 南北长度 / cos(beta)
        else:
            return 东西长度 / cos(pi / 2 - beta)

    def 直线转为测线线段(测线直线: sympy.Ray2D) -> sympy.Segment3D:
        source = 测线直线.source
        source3d = sympy.Point3D(source.x, source.y, 0)
        lineAB = sympy.Segment2D(p待测区A, p待测区B)
        lineDC = sympy.Segment2D(p待测区D, p待测区C)
        lineAD = sympy.Segment2D(p待测区A, p待测区D)
        lineBC = sympy.Segment2D(p待测区B, p待测区C)

        for i in [lineAB, lineAD, lineBC, lineDC]:
            pIntersection = 测线直线.intersection(i)
            # print(f'pIntersection {pIntersection}')
            if pIntersection and pIntersection != [source]:
                pI = pIntersection[0]
                pInter3d = sympy.Point3D(pI.x, pI.y, 0)
                return sympy.Segment3D(source3d, pInter3d)

    def 计算重叠率():
        pass

    def get各个参照物():
        res = dict()
        res["x轴向量"] = sympy.Point(1, 0, 0)
        res["y轴向量"] = sympy.Point(0, 1, 0)
        res["z轴向量"] = sympy.Point(0, 0, 1)
        res["xoy面"] = sympy.Plane((1, 0, 0), normal_vector=(0, 0, 1))
        res["xoz面"] = sympy.Plane((1, 0, 0), normal_vector=(0, 1, 0))

        res["海域中心点3d"] = sympy.Point3D(
            AD / 2,
            AB / 2,
            0 - 海域中心点水深,
        )

        res["AB向量3D"] = res["y轴向量"]
        res["AD向量3D"] = sympy.Point3D(cos(alpha), 0, -sin(alpha))

        # res["A点3d"] = (
        #     res["海域中心点3d"]
        #     + (-res["AD向量3D"].unit) * (AD / 2) / cos(alpha)
        #     + (-res["AB向量3D"].unit) * (AB / 2) / cos(alpha)
        # )
        res["A点3d"] = sympy.Point3D(
            0,
            0,
            res["海域中心点3d"].z + (AD / 2) * tan(alpha),
        )
        res["D点3d"] = res["A点3d"] + (res["AD向量3D"].unit) * AD / cos(alpha)
        res["B点3d"] = res["A点3d"] + (res["AB向量3D"].unit) * AB
        res["C点3d"] = res["D点3d"] + (res["AB向量3D"].unit) * AB

        res["坡面"] = sympy.Plane(
            res["A点3d"],
            res["D点3d"],
            res["B点3d"],
        )

        res["AB线段3d"] = sympy.Segment3D(res["A点3d"], res["B点3d"])
        res["BC线段3d"] = sympy.Segment3D(res["B点3d"], res["C点3d"])
        res["CD线段3d"] = sympy.Segment3D(res["C点3d"], res["D点3d"])
        res["DA线段3d"] = sympy.Segment3D(res["D点3d"], res["A点3d"])

        res["海底平面"] = sympy.Plane(res["D点3d"], res["z轴向量"])
        res["海深度"] = abs(float(res["A点3d"].z))

        res["AD"] = AD
        res["AB"] = AB

        return res

    return 计算覆盖宽度, 计算测线长度, 计算重叠率, 直线转为测线线段, get各个参照物


计算覆盖宽度, 计算测线长度, 计算重叠率, 直线转为测线线段, get各个参照物 = getFns(
    起始点=(nm2m(0), nm2m(0)),
    南北长度=nm2m(2),
    东西长度=nm2m(4),
    换能器开角=radians(120),
    坡度=radians(1.5),
    海域中心点水深=110,
)

In [77]:
test = get各个参照物()

x = np.array(test["坡面"].normal_vector).astype(float)
y = np.array(test["xoy面"].normal_vector).astype(float)

print(f"radians(1.5): {radians(1.5)}")
print(f"坡面坡度（弧度）: {计算两向量夹角(x, y, False)}")
print(f"坡面坡度（角度）: {计算两向量夹角(x, y, True)}")

# 验算代码构造的坡面 坡度
assert str(计算两向量夹角(x, y, False))[:10] == str(radians(1.5))[:10]

radians(1.5): 0.026179938779914945
坡面坡度（弧度）: 0.026179938779915715
坡面坡度（角度）: 1.5000000000000442


In [78]:
for i in ["A", "B", "C", "D"]:
    k = f"{i}点3d"
    print(f"{k}: {sympy.N(test[k])}")
print(f'海深度: {test["海深度"]}m')

A点3d: Point3D(0, 0, -13.0073465077316)
B点3d: Point3D(0, 3704.0, -13.0073465077316)
C点3d: Point3D(7408.0, 3704.0, -206.992653492268)
D点3d: Point3D(7408.0, 0, -206.992653492268)
海深度: 13.0073465077316m


In [79]:
print(
    [
        m2nm(test["AB线段3d"].length),
        m2nm(test["BC线段3d"].length),
        m2nm(test["CD线段3d"].length),
        m2nm(test["DA线段3d"].length),
    ]
)

# 验算代码构造的坡面 大小
assert m2nm(test["AB线段3d"].length) == 2  # 南北
assert m2nm(test["DA线段3d"].length) == 4 / cos(radians(1.5))  # 东西

[2, 4.00137116996347, 2, 4.00137116996347]


In [80]:
def 计算相对于参考点的最优测线(
    测线方向夹角: float,
    参考点: sympy.Point3D,
    开角: float = radians(120),
):
    """
    只需要考虑 测线方向夹角beta∈(0°,90°) 即可，

    因为 beta∈(0°,90°) 和 beta∈(90°,180°) 的情况是对称的

    Arguments:

    测线方向夹角 - 弧度制

    采样率 - 单位米

    Returns:

    测线线段，左边覆盖线段，右边覆盖线段
    """

    beta = 测线方向夹角

    """
    假设所有测线方案，第一条测线都要覆盖坐标原点
    那么此时判断第一条测线的方位就很重要
    只要第一条测线的方程确定了，后面的可以紧接着第一条测线继续计算
    
    如果 beta 角度大于 90° 那么需要首条测线覆盖 A 点，且从 A 点开始出发
    否则 需要覆盖 B 点，且从 B 点开始出发
    """

    def 计算首条测线出发点到极端点的距离(测线方向夹角beta):
        """
        设船只出发点为S
        """
        beta = 测线方向夹角beta

        def 计算极端角D的轨迹半径(海深度: float, 开角: float):
            """
            设测量船只的位置为 M，投影到海底平面的点为 M'
            如果 D 点刚好在角度为 beta 的测线覆盖范围边缘
            过点 D 作垂线垂直于 测线 在海底面的投影，垂足为 D'。则 M' == D'
            那么调整 beta 角度，可以得到 D' 的轨迹线，
            由于 ∠DMM'=60° 且 MM'=海深度 且 MM'⊥AM'，所以 △DMM' 可解
            """
            r = 海深度 * tan(开角 / 2)
            return r

        参考点的海深度 = abs(参考点.z)
        r = 计算极端角D的轨迹半径(参考点的海深度, 开角)
        print(f"计算极端角D的轨迹半径 {float(r)}")
        val = r / sin(beta)
        print(f"计算首条测线出发点到极端点的距离 {float(val)}")
        return val

    出发点离D点的距离 = float(计算首条测线出发点到极端点的距离(beta))
    print(f"出发点离D点的距离 {出发点离D点的距离}")
    print(f"参考点 {sympy.N(参考点)}")
    出发点 = sympy.Point3D(参考点.x - 出发点离D点的距离, 0, 0)  # ok
    # 计算出第一条测线方程
    首次测线 = 直线转为测线线段(sympy.Ray2D(出发点, angle=beta))
    print(f"首次测线 {sympy.N(首次测线)}")

In [81]:
参照物 = get各个参照物()

计算相对于参考点的最优测线(
    测线方向夹角=radians(30),
    参考点=参照物["D点3d"],
)

计算极端角D的轨迹半径 358.52179264210747
计算首条测线出发点到极端点的距离 717.0435852842152
出发点离D点的距离 717.0435852842152
参考点 Point3D(7408.0, 0, -206.992653492268)
首次测线 Segment3D(Point3D(6690.95641471578, 0, 0), Point3D(7408.0, 413.985306984537, 0))
